In [1]:
import pywt
import cv2
import numpy as np
import pandas as pd
import glob
from PIL import Image
import tensorflow as tf
import PIL
import os
import os.path
from PIL import Image
from imutils import paths
import argparse
import requests
import cv2
import os

In [2]:
# This function does the coefficient fusing according to the fusion method
def fuseCoeff(cooef1, cooef2, method):

    if (method == 'mean'):
        cooef = (cooef1 + cooef2) / 2
    elif (method == 'min'):
        cooef = np.minimum(cooef1,cooef2)
    elif (method == 'max'):
        cooef = np.maximum(cooef1,cooef2)
    else:
        cooef = []

    return cooef


# Params
FUSION_METHOD = 'mean' # Can be 'min' || 'max || anything you choose according theory



# Camera Images

def my_load_camera_data_1() :
    image_list1 = []
    for filename in glob.glob('C:/Users/MANDEEP SINGH SHERRY/Desktop/Thesis Research/Wavelet/Camera Images/testing/class 5/*.jpg'): #assuming gif
         image_list1.append(filename)
    return image_list1

# Thermal Images

def my_load_thermal_data_2() :
    image_list2 = []
    for filename in glob.glob('C:/Users/MANDEEP SINGH SHERRY/Desktop/Thesis Research/Wavelet/Thermal Images/testing/class 6/*.jpg'): #assuming gif
        image_list2.append(filename)
    return image_list2

I1=my_load_camera_data_1()
I2=my_load_thermal_data_2()

#I2 = cv2.resize(I2,I1.shape) # I do this just because i used two random images

## Fusion algo


# First: Do wavelet transform on each image
for j in range(len(I1)):
 wavelet = 'db1'
 
 img1 = cv2.imread(I1[j],0)
 img2 = cv2.imread(I2[j],0)
 cooef1 = pywt.wavedec2(img1[:,:], wavelet)
 cooef2 = pywt.wavedec2(img2[:,:], wavelet)


# Second: for each level in both image do the fusion according to the desire option
 fusedCooef = []
 for i in range(len(cooef1)-1):

    # The first values in each decomposition is the apprximation values of the top level
    if(i == 0):

        fusedCooef.append(fuseCoeff(cooef1[0],cooef2[0],FUSION_METHOD))

    else:

        # For the rest of the levels we have tupels with 3 coeeficents
        c1 = fuseCoeff(cooef1[i][0],cooef2[i][0],FUSION_METHOD)
        c2 = fuseCoeff(cooef1[i][1], cooef2[i][1], FUSION_METHOD)
        c3 = fuseCoeff(cooef1[i][2], cooef2[i][2], FUSION_METHOD)

        fusedCooef.append((c1,c2,c3))

# Third: After we fused the cooefficent we need to transfor back to get the image
 fusedImage = pywt.waverec2(fusedCooef, wavelet)

# Forth: normmalize values to be in uint8
 fusedImage = np.multiply(np.divide(fusedImage - np.min(fusedImage),(np.max(fusedImage) - np.min(fusedImage))),255)
 fusedImage = fusedImage.astype(np.uint8)

# Fith: Show image
# print(j)
 cv2.imwrite("C:/Users/MANDEEP SINGH SHERRY/Desktop/Thesis Research/Wavelet/outputDWTest/plate_bottle"+str(j)+".jpg",fusedImage)

print("Congratulations!!! fusion Completed")
f = r'c://Users/MANDEEP SINGH SHERRY//Desktop/Thesis Research/Wavelet/output'

for file in os.listdir(f):
    f_img = f+"/"+file
    img = Image.open(f_img).convert('RGB')
    img = img.resize((225,225))
    img.save(f_img)

Congratulations!!! fusion Completed
